In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima 
from sklearn.metrics import confusion_matrix, classification_report

from statsmodels.tsa.stattools import adfuller

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

from function.water_data import Water
from function.add_dam import Add_Dam

In [ ]:
wa=Water()
ad=Add_Dam()

In [ ]:
waterDF=pd.concat([wa.api_data(), wa.api_data('2017,2018,2019,2020')], axis=0)
mulgeum_df = waterDF[waterDF['총량지점명'] == '물금'].copy().set_index('일자').dropna()
geumgok_df = waterDF[waterDF['총량지점명'] == '금곡'].copy().set_index('일자').dropna()
damDF = wa.dam()

--- 분석 준비 완료: 핵심 수질 지표 ---
  총량지점명         일자   수온  수소이온농도(ph)  전기전도도(EC)  용존산소(DO)  BOD  COD  부유물질  \
0    물금 2021-01-04  4.6         7.5      437.0      13.1  1.7  5.1   2.4   
1    물금 2021-01-11  2.8         7.3      461.0      14.3  2.0  5.4   4.0   
2    물금 2021-01-26  5.1         7.2      470.0      15.6  2.1  6.0   7.6   
3    물금 2021-01-18  3.1         7.3      463.0      15.2  2.2  5.9   6.4   
4    물금 2021-02-16  6.6         8.4      451.0      12.1  2.8  6.6  12.4   

   총질소(T-N)  총인(T-P)  총유기탄소(TOC)      유량  클로로필-a  
0     3.223    0.025         3.8  29.555    12.4  
1     3.268    0.031         4.0  28.599    19.4  
2     3.416    0.026         4.6  74.897    47.3  
3     3.327    0.032         4.4  47.190    33.2  
4     3.489    0.027         5.0  72.735    50.4  
--- 분석 준비 완료: 핵심 수질 지표 ---
  총량지점명         일자   수온  수소이온농도(ph)  전기전도도(EC)  용존산소(DO)  BOD  COD  부유물질  \
0    물금 2017-01-16  3.6         7.9        391      13.7  2.4  5.9   5.2   
1    물금 2017-01-02  7.2        

In [ ]:
mulgeum_df = ad.month_dam_add_small(mulgeum_df, damDF).set_index('일자')
geumgok_df = ad.month_dam_add_small(geumgok_df, damDF).set_index('일자')

In [ ]:
mxtrain,mxtest,mytrain,mytest=ad.log_scale(mulgeum_df)
gxtrain,gxtest,gytrain,gytest=ad.log_scale(geumgok_df)

In [ ]:
mulgeum_model = auto_arima(
    mytrain, 
    exogenous=mxtrain,      # 외부 변수 (방류량, 강수량 등)
    start_p=1, start_q=1,
    max_p=3, max_q=3,      
    m=12,                  # 계절성 주기 (월별 데이터면 12)
    seasonal=True,         # 계절성 고려 여부
    d=1,                # d값을 자동으로 찾음
    trace=True,            # 중간 과정을 출력 (학습 상황 확인용)
    error_action='ignore',  
    suppress_warnings=True, 
    stepwise=True
)


Performing stepwise search to minimize aic
 ARIMA(1,1,1)(1,0,1)[12] intercept   : AIC=inf, Time=0.69 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=161.058, Time=0.03 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=157.585, Time=0.08 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=inf, Time=0.41 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=159.137, Time=0.00 sec
 ARIMA(1,1,0)(0,0,0)[12] intercept   : AIC=158.228, Time=0.02 sec
 ARIMA(1,1,0)(2,0,0)[12] intercept   : AIC=154.666, Time=0.24 sec
 ARIMA(1,1,0)(2,0,1)[12] intercept   : AIC=inf, Time=1.36 sec
 ARIMA(1,1,0)(1,0,1)[12] intercept   : AIC=inf, Time=0.53 sec
 ARIMA(0,1,0)(2,0,0)[12] intercept   : AIC=162.120, Time=0.14 sec
 ARIMA(2,1,0)(2,0,0)[12] intercept   : AIC=151.812, Time=0.45 sec
 ARIMA(2,1,0)(1,0,0)[12] intercept   : AIC=156.354, Time=0.10 sec
 ARIMA(2,1,0)(2,0,1)[12] intercept   : AIC=inf, Time=1.32 sec
 ARIMA(2,1,0)(1,0,1)[12] intercept   : AIC=inf, Time=0.49 sec
 ARIMA(3,1,0)(2,0,0)[12] intercept   : AIC=152.671, Time=

In [ ]:
print(mulgeum_model.summary())

                                     SARIMAX Results                                      
Dep. Variable:                                  y   No. Observations:                   76
Model:             SARIMAX(0, 1, 2)x(1, 0, 2, 12)   Log Likelihood                 -59.127
Date:                            Tue, 30 Dec 2025   AIC                            130.253
Time:                                    09:41:54   BIC                            144.158
Sample:                                01-01-2017   HQIC                           135.805
                                     - 04-01-2023                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.7776      0.146     -5.325      0.000      -1.064      -0.491
ma.L2         -0.1958      0.131   

<h1>결론</h1>

독립 변수의 측정 한계: 대부분의 독립변수의 p-value가 0.5보다 크며, 그래서 p-value(통계적 유의성) 미확보로 인해 독립변수는 시계열데이터에 비해서 유의미하지 않다고 결론이 내려졌다. 이 원인으로는 다양한 변수들은 서로 어는정도 상관관계가 있는 데이터라는 특성이 있고 이로 인해 변수 간의 높은 상관관계(다중공선성?) 로인해 서로가 영향도를 떨어트려서 p-value가 높아지는 경향을 볼 수 있다.

시차의 부재: 다양한 변수들, 예를들면 수온과 총질소 같은 값들은 즉각적으로 클로로필수치에 영향을 미치지 않고 어는 정도의 시간이 지난 후로 값에 영향을 미치는 시간지연이 있다. 이를 반영하기 위해, 달별로 평균을 구하는 데이터에서 다양한 종속변수를 넣어서 정확도를 올리기에는 한계가 있다는 사실을 알 수 있다.

향후 방향: 조류의 증식은 환경변화에 즉각적인 변화를 반영하지는 못한다. 보통 조류는 환경변화 후 3~10일 정도의 시차가 적용되고 변화하는데 월 평균 데이터는 이 변화를 적용하기 힘들기에 월 평균 데이터는 경향예측을 위주로 (sarima) 사용해야 한다. 그렇지만 특정 독립 변수 1개만 남기며 전월 데이터를 적용시키는 시차를 발생시킨다면 유용할 가능성이 있다.(수온, 유량)

분석 뱡향: 환경 변화가 클로로필-a수치에 영향을 미치는 시차의 평균을 구하기 위해서 몇일 전의 수온, 유량들이 당일날의 클로로필-a에 영향을 어는정도 미치는지 분석이 필요하다고 생각한다. 그래서 최적의 시차를 구할 필요성이 있어 보인다.